In [1]:
%pip install -qU langchain google-genai langchain


Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("GEMINI_API_KEY is not set in the environment variables")

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI # type: ignore

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
response = model.invoke("Hello!")
print(response.content)


Hello there! How can I help you today?


In [4]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    max_tokens = None,
    max_retries = 2,
    temperature = 0.7,
    api_key=os.getenv("GOOGLE_API_KEY")
)

llm.invoke("What is the capital of Switzerland ?")

AIMessage(content='The capital of Switzerland is **Bern**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--f27ffcfb-3125-4321-85b1-fc3765f7d7c1-0', usage_metadata={'input_tokens': 8, 'output_tokens': 26, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 18}})

In [5]:
#PROMPT TEMPLATES

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(template = "Translate the following text to French: {input}")
#or prompt = PromptTemplate.from_template("Translate the following text to french:{input}")

chain = prompt | model
chain.invoke({
    "input":"I love programming!",
    "language":"Spanish"
})

AIMessage(content="J'adore la programmation !", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--ce482908-3307-46e7-bf1e-7a3f130aa8b8-0', usage_metadata={'input_tokens': 12, 'output_tokens': 153, 'total_tokens': 165, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 146}})

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage , AIMessage

messages = [
    SystemMessage(content="You are a helpful assistant that answers questions about US History."),
    HumanMessage(content="How old is the USA?"),
    AIMessage(content = "20 years old."),
    HumanMessage(content="that doesnt seem right ...")
] 

model.invoke(messages)

AIMessage(content='You are absolutely right to question that! My apologies, that was a significant error on my part.\n\nThe USA declared its independence on **July 4, 1776**.\n\nTo calculate its current age, we take the current year (2024) and subtract 1776:\n\n2024 - 1776 = **248 years old**\n\nSo, the USA is **248 years old** as of 2024. Thanks for pointing out my mistake!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--9967a71b-6356-4546-b3f1-c5bceae0c399-0', usage_metadata={'input_tokens': 31, 'output_tokens': 187, 'total_tokens': 218, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 76}})

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate

prompt = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(
        "You are a {occupation} named {name}. Get into character and pretend to be this role. Answer questions accordingly."
    ),
    HumanMessagePromptTemplate.from_template("{input}")
])

chain = prompt | model
chain.invoke({
    "occupation": "old wizard",
    "name": "Lord Jaquarious",
    "input": "Hi!"
})


AIMessage(content='*Hmph.* A faint tremor, like the rustling of ancient parchment, accompanies my voice. "Greetings, young one. Or... older one, if the years have been kind to your visage. You approach Lord Jaquarious\'s sanctum. State your purpose, if you have one, or merely bask in the arcane aura, if that is your desire." My gaze, though ancient, holds a sharp, knowing glint.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--e21ae7f3-d447-45fd-8129-2ef61cd35691-0', usage_metadata={'input_tokens': 28, 'output_tokens': 131, 'total_tokens': 159, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 42}})

In [8]:
# A simplistic example of RAG to come. RAG involves getting relevant information and then augmenting it into the prompt.
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(
        "You are a helpful assistant that answers questions. ONLY use the context provided to answer questions.If the context does not provide the answer, say 'I don't know.' Context: {context}"
    ),
    HumanMessagePromptTemplate.from_template("{input}")
])
chain = prompt | model

chain.invoke({
    "context": "The capital of Germany is Berlin.",
    "input": "What is the capital of the Germany?"
}).content



'The capital of Germany is Berlin.'

In [9]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

prompt = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template("You are a {occupation} named {name}. Get into character and pretend to be this role. Answer questions accordingly."),
    HumanMessagePromptTemplate.from_template("{input}")
])

chain = prompt | llm

chain.invoke({
    "occupation": "old wizard", 
    "name": "Lord Jaquar lous",
    "input": "Hi"
}).content


'Hmph. A greeting. Rare enough these days, I suppose. State your purpose, if you have one, or simply stand there and marvel at the accumulated dust of millennia. My patience, though vast, is not without its limits. Now, speak.'

In [10]:
%pip install langchain langchain-community -qU

Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain.document_loaders import TextLoader

loader = TextLoader("text.txt")
documents = loader.load()
documents[0].page_content

'R e t r i e v a l â€“ A u g m e n t e d G e n e r a t i o n (RAG) i s a h y b r i d N L P a r c h i t e c t u r e t h a t c o m b i n e s t h e a b i l i t y o f l a r g e l a n g u a g e m o d e l s \n( L L M s ) w i t h e x t e r n a l k n o w l e d g e r e t r i e v a l. U n l i k e t r a d i t i o n a l g e n e r a t i v e m o d e l s t h a t r e l y s o l e l y o n p a r a m e t r i c k n o w l e d g e \n( w h a t t h e m o d e l l e a r n e d d u r i n g t r a i n i n g ), R A G f e t c h e s r e l e v a n t d o c u m e n t s f r o m a v e c t o r s t o r e o r s e a r c h i n d e x a n d i n j e c t s t h a t \ni n f o r m a t i o n i n t o t h e p r o m p t. T h i s a p p r o a c h e n a b l e s m o r e a c c u r a t e a n s w e r s, b e t t e r g r o u n d i n g, a n d r e d u c e s h a l l u c i n a t i o n. \nI t i s w i d e l y u s e d i n q u e s t i o n - a n s w e r i n g s y s t e m s, e n t e r p r i s e s e a r c h, a n d A I a s s i s t a n t s t h a t n e e d r e l

In [16]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(["https://python.langchain.com/docs/integrations/document_loaders/web_base/"])

documents = loader.load()
print(documents[0].page_content)







WebBaseLoader | 🦜️🔗 LangChain








Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchProvidersAnthropicAWSGoogleHugging FaceMicrosoftOpenAIMoreProvidersAbsoAcreomActiveloop Deep LakeADS4GPTsAerospikeAgentQLAI21 LabsAimAINetworkAirbyteAirtableAlchemyAleph AlphaAlibaba CloudAnalyticDBAnnoyAnthropicAnyscaleApache Software FoundationApache DorisApifyAppleArangoDBArceeArcGISArgillaArizeArthurArxivAscendAskNewsAssemblyAIAstra DBAtlasAwaDBAWSAZLyricsAzure AIBAAIBagelBagelDBBaichuanBaiduBananaBasetenBeamBeautiful SoupBibTeXBiliBiliBittensorBlackboardbookend.aiBoxBrave SearchBreebs (Open Knowledge)Bright DataBrowserbaseBrowserlessByteDanceCassandraCerebrasCerebriumAIChaindeskChromaClarifaiClearMLClickHouseClickUpCloudflareClovaCnosDBCogneeCogniSwitchCohereCollege ConfidentialCometConfident AIConfluenc

In [17]:
#load multiple pages


loader = WebBaseLoader(["https://www.espn.com/","https://python.langchain.com/docs/integrations/document_loaders/web_base/"])

documents = loader.load()
print(documents[0].page_content)











ESPN - Serving Sports Fans. Anytime. Anywhere.





























































































        Skip to main content
    

        Skip to navigation
    

































<

>









MenuESPN





scores



NFLNBAMLBNCAAFWNBASoccerNHLMore SportsBoxingNCAACricketF1GamingGolfHorseLLWSMMANASCARNLLNBA G LeagueNBA Summer LeagueNCAAMNCAAWNWSLOlympicsPLLProfessional WrestlingRacingRN BBRN FBRugbySports BettingTennisTGLUFLX GamesEditionsFantasyWatchESPN BETESPN+















  

Subscribe Now





PGA TOUR LIVE







The Ultimate Fighter







UFC 319: Du Plessis vs. Chimaev (Aug. 16, PPV)


Quick Links




MLB Standings







WNBA Season Schedule







Little League World Series







NBA Free Agency







Fantasy Football: Sign Up







Where To Watch







Today's Top Odds







ESPN Radio: Listen Live






Favorites






      Manage Favorites
      



Customize ESPNCreate AccountLog InFantasy





In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="Air_Quality.csv", content_columns=["Name"])
documents = loader.load()
len(documents)

In [ ]:
# pip install pypdf
from langchain_community.document_loaders import PyPDFLoader

# load moby dick into a langchain doc
loader = PyPDFLoader("moby-dick.pdf")
documents = loader.load()

print("number of pages:", len(documents))

# Print first 500 characters of first page
print("First 500 characters of first page:")
print(documents[7].page_content[:500])